<a href="https://colab.research.google.com/github/benmancilla/EDA_FundamentosML_BenjaminMancilla/blob/Revision/Revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos las librerías necesarias para el análisis de datos.

In [4]:
import numpy as np # Programación vectorial
import matplotlib.pyplot as plt # Visualización
import pandas as pd # Estructura y manipulación de datos
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

Subimos el data center.

In [51]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
data = pd.read_csv(name, sep=",", encoding='latin-1') # Changed encoding to 'latin-1'

data.head()

Saving data.csv.zip to data.csv (5).zip


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Exploracion inicial de datos

In [52]:
#información general
print("\nInformación general del dataset:")
data.info()
# Descripción estadística básica
print("\nEstadísticas descriptivas básicas:")
data.describe()


Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

Estadísticas descriptivas básicas:


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Buscamos datos nulos

In [53]:
print("\nTotal de valores nulos por columna:")
#Ver valores nulos
null_values = data.isnull().sum()
print(null_values[null_values > 0])


Total de valores nulos por columna:
Description      1454
CustomerID     135080
dtype: int64


Eliminamos datos nulos que obstruyan el análisis, en este caso los datos nulos de la columna "Description"

In [54]:
# Eliminar filas donde la columna 'Description' tenga valores nulos
data = data.dropna(subset=['Description'])
print("Valores nulos en 'Description' después de la limpieza:", data['Description'].isnull().sum())

Valores nulos en 'Description' después de la limpieza: 0


Identificación y clasificación de variables.

In [55]:
# Clasificación de variables
print("\n--- Clasificación de variables ---")

# Variables numéricas (cuantitativas)
numericas_continuas = ['UnitPrice']  # Precio unitario
numericas_discretas = ['Quantity', 'CustomerID']  # Cantidad de productos y ID del cliente

# Variables categóricas (cualitativas)
categoricas_nominales = ['InvoiceNo', 'StockCode', 'Description', 'Country']  # Factura, código del producto, descripción y país

# Variables temporales
temporales = ['InvoiceDate']  # Fecha de la compra

# Imprimir los tipos de las variables
print("Variables numéricas continuas:", numericas_continuas)
print("Variables numéricas discretas:", numericas_discretas)
print("Variables categóricas nominales:", categoricas_nominales)
print("Variables temporales:", temporales)

# Convertir las variables categóricas y temporales al tipo correcto
for col in categoricas_nominales:
    data[col] = data[col].astype('category')

# Convertir la variable de fecha a tipo datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')

# Verificar los tipos de datos después de la conversión
print("\nTipos de datos después de la conversión:")
print(data.dtypes)


--- Clasificación de variables ---
Variables numéricas continuas: ['UnitPrice']
Variables numéricas discretas: ['Quantity', 'CustomerID']
Variables categóricas nominales: ['InvoiceNo', 'StockCode', 'Description', 'Country']
Variables temporales: ['InvoiceDate']

Tipos de datos después de la conversión:
InvoiceNo            category
StockCode            category
Description          category
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country              category
dtype: object


## Análisis estadístico descriptivo

1.Ticket promedio

In [56]:
ticket_promedio = data.groupby('InvoiceNo').apply(lambda x: (x['Quantity'] * x['UnitPrice']).sum()).mean()
# Imprimir resultado
print(f"Ticket promedio: £{ticket_promedio:.2f}")

if ticket_promedio > 100:
    conclusion = "Los clientes tienden a hacer compras de alto valor."
else:
    conclusion = "Los clientes tienden a hacer compras de bajo valor."

print(conclusion)

Ticket promedio: £398.75
Los clientes tienden a hacer compras de alto valor.


2.Productos mas vendidos

In [63]:
# Agrupar por producto (Description) y sumar la cantidad vendida
productos_mas_vendidos = data.groupby('Description')['Quantity'].sum().sort_values(ascending=False)

# Mostrar los 10 productos más vendidos
print("Top 10 productos más vendidos: ")
print(productos_mas_vendidos.head(10)()
print("Los primeros lugares entre los productos mas vendidos no pueden mostrar una tendencia de la clientela hacia productos de un estilo retro, ya sea con fines decorativos, coleccionables o de regalo")

Top 10 productos más vendidos: 
Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53847
JUMBO BAG RED RETROSPOT               47363
ASSORTED COLOUR BIRD ORNAMENT         36381
POPCORN HOLDER                        36334
PACK OF 72 RETROSPOT CAKE CASES       36039
WHITE HANGING HEART T-LIGHT HOLDER    35317
RABBIT NIGHT LIGHT                    30680
MINI PAINT SET VINTAGE                26437
PACK OF 12 LONDON TISSUES             26315
PACK OF 60 PINK PAISLEY CAKE CASES    24753
Name: Quantity, dtype: int64
Los primeros lugares entre los productos mas vendidos no pueden mostrar una tendencia /rb de la clientela hacia productos de un estilo retro, ya sea con fines decorativos, coleccionables o de regalo
